In [ ]:
# math
import numpy as np
import pandas as pd

from experiments_2022 import IMAGE_PATH
from experiments_2022.zone_level_analysis import (
    base,
    cleaning,
    viz,
    regression_functions,
    clustering,
)
from experiments_2022.datasets import (
    load_weather,
    load_building,
    pull_from_dataset,
)

from utils import (
    PROJECTS_2021,
    PROJECTS_2022,
    NO_WEEKENDS,
    SUMMER_START_2022,
    SUMMER_END_2022,
    REACTIVE_THRESH,
    SINGLE_PLOT_LEGEND_SIZE,
    SINGLE_PLOT_TXT_SIZE,
    RESPONSE_COLORS,
    run_building_regressions,
    run_equip_regressions,
    add_vertical_boxes,
)

In [ ]:
REACTIVE_ZONES = {}
for project in PROJECTS_2022:
    rzs = pd.read_csv(f"./dicts_dfs/reactive/{project}.csv").set_index("Unnamed: 0")
    rzs.index.name = None
    REACTIVE_ZONES[project] = rzs

# Change in cooling

In [ ]:
T_2022 = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=False,
    SI_units=True,
)["temperature"]

In [ ]:
T_2021 = cleaning.clean_df(
    df=load_weather("2021")["temperature"].to_frame(),
    this_var="weather-oat",
    start_date=pd.Timestamp("05-01-2021"),
    end_date=pd.Timestamp("10-01-2021"),
    only_business_hours=True,
    no_weekends=False,
    SI_units=True,
)["temperature"]

In [ ]:
# fig

In [ ]:
cooling_df_2022 = cleaning.clean_by_column(
    df=load_building("2022", "C") * base.MW_PER_TON * 1000,  # kWh
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
)

cooling_df_2022 = cleaning.clean_df(
    df=cooling_df_2022[PROJECTS_2022],
    this_var="building-cooling",
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    only_business_hours=True,
    no_weekends=False,
)
cooling_2022 = {}
for project in PROJECTS_2022:
    cooling_2022[project] = cooling_df_2022[project].to_frame()
    for day in ["06-23-2022"]:
        cooling_2022[project].loc[day, :] = np.nan

In [ ]:
fig = viz.make_time_series(
    load_building("2022", "C").loc[SUMMER_START_2022:SUMMER_END_2022, :],
    y_axis_title="Cooling (tons)",
)

In [ ]:
fig = viz.make_time_series(cooling_df_2022, y_axis_title="Cooling (kWh)")

In [ ]:
# fig

In [ ]:
cooling_df_2021 = cleaning.clean_by_column(
    df=load_building("2021", "C")[PROJECTS_2022] * base.MW_PER_TON * 1000,  # kWh
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
)
cooling_df_2021 = cleaning.clean_df(
    df=cooling_df_2021,
    this_var="building-cooling",
    start_date=pd.Timestamp("05-01-2021"),
    end_date=pd.Timestamp("10-01-2021"),
    only_business_hours=True,
    no_weekends=False,
)
cooling_2021 = {}
for project in PROJECTS_2021:
    cooling_2021[project] = cooling_df_2021[project].to_frame()

In [ ]:
fig = viz.make_time_series(
    load_building("2021", "C").loc[
        pd.Timestamp("05-01-2021") : pd.Timestamp("10-01-2021"), :
    ],
    y_axis_title="Cooling (tons)",
)

In [ ]:
fig = viz.make_time_series(cooling_df_2021, y_axis_title="Cooling (kwh)")

In [ ]:
(
    deltas_cooling,
    deltas_high_cooling,
    deltas_low_cooling,
    percent_summary,
    fig,
) = run_building_regressions(
    cooling_2022,
    T_2022,
    mode="Percent Change",
    summary_statistic="Mean",
    y_axis_title="Average Cooling Demand<br>(kW/1000m2)",
    use_raw=False,
)

In [ ]:
(
    deltas_cooling_2021,
    deltas_high_cooling_2021,
    deltas_low_cooling_2021,
    percent_summary,
    fig,
) = run_building_regressions(
    cooling_2021,
    T_2021,
    year="2021",
    mode="Percent Change",
    summary_statistic="Mean",
    y_axis_title="Average Cooling Demand<br>(kW/1000m2)",
    use_raw=False,
)

In [ ]:
deltas_list = [deltas_cooling, deltas_high_cooling, deltas_low_cooling]
for i, df in enumerate(deltas_list):
    df.columns = ["CSP = 24.4°C (2022)", "CSP = 25.5°C (2022)"]
    df["CSP = 24.4°C (2021)"] = np.nan
    df = df[["CSP = 25.5°C (2022)", "CSP = 24.4°C (2022)", "CSP = 24.4°C (2021)"]]
    deltas_list[i] = df
deltas_cooling, deltas_high_cooling, deltas_low_cooling = deltas_list

In [ ]:
# previously reported results
beta = {
    "OFF-1": -0.24,  # previously CONF-1
    "OFF-3": -0.33,  # previously OFF-2
    "OFF-4": -0.14,  # previously LIB-3
    "OFF-6": -0.23,  # previously OFF-4
    "LAB-1": -0.044,  # previously LAB-5
    "LAB-3": -0.034,  # previously LAB-6
}

err = {
    "OFF-1": 0.048,  # previously CONF-1
    "OFF-3": 0.028,  # previously OFF-2
    "OFF-4": 0.031,  # previously LIB-3
    "OFF-6": 0.022,  # previously OFF-4
    "LAB-1": 0.017,  # previously LAB-5
    "LAB-3": 0.0092,  # previously LAB-6
}

for project in beta:
    deltas_cooling.loc[project, "CSP = 24.4°C (2021)"] = 100 * (
        np.exp(beta[project]) - 1
    )
    deltas_high_cooling.loc[project, "CSP = 24.4°C (2021)"] = (
        100 * (np.exp(beta[project] - 1.96 * err[project]) - 1)
        - deltas_cooling.loc[project, "CSP = 24.4°C (2021)"]
    )
    deltas_low_cooling.loc[project, "CSP = 24.4°C (2021)"] = deltas_cooling.loc[
        project, "CSP = 24.4°C (2021)"
    ] - 100 * (np.exp(beta[project] + 1.96 * err[project]) - 1)

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=deltas_cooling,
    y_error_up_data=deltas_high_cooling,
    y_error_down_data=deltas_low_cooling,
    marker_legend={
        "color": {
            "CSP = 24.4°C (2021)": "Blue",
            "CSP = 24.4°C (2022)": "Blue",
            "CSP = 25.5°C (2022)": "Black",
        },
        "opacity": {
            "CSP = 24.4°C (2021)": 0.5,
            "CSP = 24.4°C (2022)": 1,
            "CSP = 25.5°C (2022)": 1,
        },
    },
    y_axis_title="Percent Change in<br>Cooling Demand [%]",
    point_start=(1 / 4),
    offset_delta=(1 / 4),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_2022)],
    y_range=[-60, 30],
    error_thickness=2.5,
    whisker_len=8,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_2022) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureB1.png")

# Fraction of responding zones

In [ ]:
reactive_zones_all = pd.DataFrame(
    index=PROJECTS_2022,
    columns=["CSP = 25.5°C (2022)", "CSP = 24.4°C (2022)", "CSP = 24.4°C (2021)"],
)
reactive_zones_all.loc["TOTAL", :] = 0

## CSP = 78F, 2022

In [ ]:
total = 0
for project in PROJECTS_2022:
    ser = REACTIVE_ZONES[project]
    reactive_zones_all.loc[project, "CSP = 25.5°C (2022)"] = len(
        ser[ser == 0].dropna()
    ) / len(ser)
    reactive_zones_all.loc["TOTAL", "CSP = 25.5°C (2022)"] += len(
        ser[ser == 0].dropna()
    )
    total += len(ser)
reactive_zones_all.loc["TOTAL", "CSP = 25.5°C (2022)"] /= total

## CSP = 76F, 2022

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    only_business_hours=True,
    no_weekends=False,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
)["temperature"]

tloads = pull_from_dataset("2022", PROJECTS_2022, "zone-tloads")
tloads = cleaning.clean_dfs(
    dfs=tloads,
    this_var="zone-tloads",
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    remove_FCUs=False,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
)
zonal_schedules = {}
for project in PROJECTS_2022:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2022", freq="hourly", df=tloads[project]
    )

df_filter = cleaning.create_sp_filter(zonal_schedules, sps=[74], reverse_filter=False)
control_tloads = cleaning.clean_dfs(
    dfs=tloads, only_business_hours=False, no_weekends=False, df_filter=df_filter
)

control_tloads = base.run_passive_test_on_dfs(
    dfs=control_tloads,
    this_test="Mean",
    col_name="Average Zonal Load [%]<br>Control Days",
)

In [ ]:
(
    deltas_76_tloads_2022,
    deltas_low_76_tloads_2022,
    deltas_high_76_tloads_2022,
    deltas_78_tloads,
    deltas_low_78_tloads,
    deltas_high_78_tloads,
) = run_equip_regressions(
    tloads,
    T,
    "Absolute Change",
)

In [ ]:
reactive_zones = clustering.run_1D_clustering_on_dict(
    deltas_76_tloads_2022,
    slices=[REACTIVE_THRESH["Pos"], REACTIVE_THRESH["Neg"]],
    mapping={2: 0, 1: 1, 0: 3},
)

In [ ]:
for project in PROJECTS_2022:
    # small change in tload, but remained high
    these_rzs = reactive_zones[project].iloc[:, 0]
    these_rzs = list(these_rzs[these_rzs == 1].index)
    these_control = control_tloads[project].iloc[:, 0]
    these_control = list(
        these_control[these_control >= REACTIVE_THRESH["High Constant"]].index
    )
    correct = list(set(these_rzs).intersection(set(these_control)))
    reactive_zones[project].loc[correct, :] = 2

for project in PROJECTS_2022:
    # in heating
    these_control = control_tloads[project].iloc[:, 0]
    these_control = list(
        these_control[these_control <= REACTIVE_THRESH["Heating"]].index
    )
    reactive_zones[project].loc[these_control, :] = 4  # np.nan

for project in PROJECTS_2022:
    reactive_zones[project] = reactive_zones[project].dropna()

In [ ]:
total = 0
for project in PROJECTS_2022:
    ser = reactive_zones[project]
    reactive_zones_all.loc[project, "CSP = 24.4°C (2022)"] = len(
        ser[ser == 0].dropna()
    ) / len(ser)
    reactive_zones_all.loc["TOTAL", "CSP = 24.4°C (2022)"] += len(
        ser[ser == 0].dropna()
    )
    total += len(ser)
reactive_zones_all.loc["TOTAL", "CSP = 24.4°C (2022)"] /= total

## CSP = 76F, 2021

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2021")["temperature"].to_frame(),
    this_var="weather-oat",
    only_business_hours=True,
    no_weekends=False,
    start_date=pd.Timestamp("05-01-2021"),
    end_date=pd.Timestamp("10-01-2021"),
    SI_units=True,
)["temperature"]

tloads = pull_from_dataset("2021", PROJECTS_2021, "zone-tloads")
tloads = cleaning.clean_dfs(
    dfs=tloads,
    this_var="zone-tloads",
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    remove_FCUs=False,
    start_date=pd.Timestamp("05-01-2021"),
    end_date=pd.Timestamp("10-01-2021"),
)

zonal_schedules = {}
for project in PROJECTS_2021:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2021", freq="hourly", df=tloads[project]
    )

df_filter = cleaning.create_sp_filter(zonal_schedules, sps=[74], reverse_filter=False)
control_tloads = cleaning.clean_dfs(
    dfs=tloads, only_business_hours=False, no_weekends=False, df_filter=df_filter
)

control_tloads = base.run_passive_test_on_dfs(
    dfs=control_tloads,
    this_test="Mean",
    col_name="Average Zonal Load [%]<br>Control Days",
)

In [ ]:
(
    deltas_76_tloads_2021,
    deltas_low_76_tloads_2021,
    deltas_high_76_tloads_2021,
) = run_equip_regressions(tloads, T, "Absolute Change", year="2021")

In [ ]:
reactive_zones = clustering.run_1D_clustering_on_dict(
    deltas_76_tloads_2021,
    slices=[REACTIVE_THRESH["Pos"], REACTIVE_THRESH["Neg"]],
    mapping={2: 0, 1: 1, 0: 3},
)

In [ ]:
for project in PROJECTS_2021:
    # small change in tload, but remained high
    these_rzs = reactive_zones[project].iloc[:, 0]
    these_rzs = list(these_rzs[these_rzs == 1].index)
    these_control = control_tloads[project].iloc[:, 0]
    these_control = list(
        these_control[these_control >= REACTIVE_THRESH["High Constant"]].index
    )
    correct = list(set(these_rzs).intersection(set(these_control)))
    reactive_zones[project].loc[correct, :] = 2

for project in PROJECTS_2021:
    # in heating
    these_control = control_tloads[project].iloc[:, 0]
    these_control = list(
        these_control[these_control <= REACTIVE_THRESH["Heating"]].index
    )
    reactive_zones[project].loc[these_control, :] = 4  # np.nan

for project in PROJECTS_2021:
    reactive_zones[project] = reactive_zones[project].dropna()

In [ ]:
total = 0
for project in PROJECTS_2021:
    ser = reactive_zones[project]
    reactive_zones_all.loc[project, "CSP = 24.4°C (2021)"] = len(
        ser[ser == 0].dropna()
    ) / len(ser)
    reactive_zones_all.loc["TOTAL", "CSP = 24.4°C (2021)"] += len(
        ser[ser == 0].dropna()
    )
    total += len(ser)
reactive_zones_all.loc["TOTAL", "CSP = 24.4°C (2021)"] /= total

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=reactive_zones_all,
    marker_legend={
        "color": {
            "CSP = 24.4°C (2021)": RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            "CSP = 24.4°C (2022)": RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
            "CSP = 25.5°C (2022)": RESPONSE_COLORS[
                f"Reduced zonal load {abs(REACTIVE_THRESH['Neg'])}% or more"
            ],
        },
        "opacity": {
            "CSP = 24.4°C (2021)": 0.5,
            "CSP = 24.4°C (2022)": 1,
            "CSP = 25.5°C (2022)": 1,
        },
        "shape": {
            "CSP = 24.4°C (2021)": "x",
            "CSP = 24.4°C (2022)": "x",
            "CSP = 25.5°C (2022)": "circle",
        },
    },
    y_axis_title=f"Fraction of Zones<br>Reducing Zonal Load<br>{-REACTIVE_THRESH['Neg']}% or More [Unitless]",
    point_start=(1 / 4),
    offset_delta=(1 / 4),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_2022) + 1)],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_2022) + 1],
    y_range=[0, 1],
    error_thickness=2.5,
    whisker_len=8,
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)
fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_2022) + 1 + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureB2.png")